# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 12/10/2024

This notebook will convert the (potentially) multi-valued results of a point-by-point strain refinement process to a single-valued 'TensorMap' with many useful export formats, like H5, Paraview XDMF, and MTEX CTF.

In [ ]:
import os

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
import numba
import numpy as np
import scipy.ndimage as ndi
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize

from ImageD11.grain import grain
from ImageD11 import unitcell
import ImageD11.sinograms.dataset
from ImageD11.sinograms.point_by_point import PBPMap, nb_inv, PBPRefine
from ImageD11.sinograms.tensor_map import TensorMap
from ImageD11.nbGui import nb_utils as utils

%matplotlib ipympl

In [ ]:
# USER: Pass path to dataset file

dset_file = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

ds = ImageD11.sinograms.dataset.load(dset_file)
   
sample = ds.sample
dataset = ds.dsname
rawdata_path = ds.dataroot
processed_data_root_dir = ds.analysisroot

print(ds)
print(ds.shape)

In [ ]:
# load phases from parameter file

ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

In [ ]:
# now let's select a phase to index from our parameters json
phase_str = 'Fe'

ref_ucell = ds.phases.unitcells[phase_str]

print(ref_ucell.lattice_parameters, ref_ucell.spacegroup)

In [ ]:
# import refinement manager

refine = PBPRefine.from_h5(ds.refmanfile)

In [ ]:
# now we inspect the results of the refined map

# plot a histogram of unique peaks per ubi

refine.refinedmap.plot_nuniq_hist()

In [ ]:
# choose the minimum number of peaks you want a pixel to have to be counted

min_unique = 400

refine.refinedmap.choose_best(min_unique)

# refine.refinedmap.choose_best(min_unique)

In [ ]:
# let's plot the result of your choice

refine.refinedmap.plot_best(min_unique)

In [ ]:
# look at the fancy strain results
# this is from refine.refinedmap.best_eps

fig, axs = plt.subplots(3,3, sharex=True, sharey=True, layout='constrained', figsize=(10,10))

cmap = cm.get_cmap('RdBu_r')
normalizer = Normalize(-1e-3, 1e-3)
im = cm.ScalarMappable(norm=normalizer, cmap=cmap)

for i in range(3):
    for j in range(3):
        axs[i,j].imshow(refine.refinedmap.best_eps[:, :, i, j], origin="lower", cmap=cmap, norm=normalizer)
        axs[i,j].set_title(f'eps_{i+1}{j+1}')
fig.supxlabel('< Lab Y axis')
fig.supylabel('Lab X axis')
fig.colorbar(im, ax=axs.ravel().tolist())
plt.show()

In [ ]:
# now make a TensorMap from our refined map so we can plot and export

# first let's work out what phase we have
phases = {0: ref_ucell}

# let's make a phase id map from our pbpmap
phase_ids = TensorMap.recon_order_to_map_order(np.where(refine.refinedmap.best_nuniq > min_unique, 0, -1))

# reshape the fancy strain map too
eps_sample = TensorMap.recon_order_to_map_order(refine.refinedmap.best_eps)

tmap = TensorMap.from_pbpmap(refine.refinedmap, steps=(1, ds.ystep, ds.ystep), phases=phases)
tmap['phase_ids'] = phase_ids
tmap['eps_sample'] = eps_sample

In [ ]:
# Plot the tensormap unique peaks

tmap.plot('nuniq')

In [ ]:
# compute the IPF colours from the UBIs and phase

tmap.get_ipf_maps()

In [ ]:
tmap.plot('ipf_x')
tmap.plot('ipf_y')
tmap.plot('ipf_z')

In [ ]:
fig, axs = plt.subplots(3,3, sharex=True, sharey=True, layout='constrained', figsize=(10,10))

cmap = cm.get_cmap('RdBu_r')
normalizer = Normalize(-1e-3, 1e-3)
im = cm.ScalarMappable(norm=normalizer, cmap=cmap)

for i in range(3):
    for j in range(3):
        axs[i,j].imshow(tmap.eps_sample[0, ..., i, j], origin="lower", cmap=cmap, norm=normalizer)
        axs[i,j].set_title(f'eps_{i+1}{j+1}')
fig.supxlabel('Lab X axis --->')
fig.supylabel('Lab Y axis --->')
fig.colorbar(im, ax=axs.ravel().tolist())
plt.show()

In [ ]:
# look at unit cells - mean of a, b, c for cubic for now

fig, ax = plt.subplots()
ax.hist(tmap.unitcell[0, :, :, :3].mean(axis=-1).ravel(), bins=1000)
ax.set_xlabel('unitcell of pixel')
plt.show()

In [ ]:
# mean unitcell?

print(np.nanmean(tmap.unitcell[0, :, :, :3]))

In [ ]:
# trigger calculation of all the maps

eul = tmap.euler

In [ ]:
# save the refined TensorMap to disk

tmap.to_h5(os.path.join(ds.analysispath, 'pbp_tensormap_refined.h5'))
tmap.to_paraview(os.path.join(ds.analysispath, 'pbp_tensormap_refined.h5'))

In [ ]:
# you can also do an MTEX export if you like:

ctf_path = os.path.join(ds.analysispath, 'pbp_tensormap_refined.ctf')

tmap.to_ctf_mtex(ctf_path, z_index=0)

In [ ]:
ds.save()

In [ ]:
if 1:
    raise ValueError("Change the 1 above to 0 to allow 'Run all cells' in the notebook")

In [ ]:
# We can run the below cell to do this in bulk for many samples/datasets
# by default this will do all samples in sample_list, all datasets with a prefix of dset_prefix
# you can add samples and datasets to skip in skips_dict

skips_dict = {
    "FeAu_0p5_tR_nscope": ["top_-50um", "top_-100um"]
}

dset_prefix = "top"

sample_list = ["FeAu_0p5_tR_nscope"]
    
samples_dict = utils.find_datasets_to_process(ds.dataroot, skips_dict, dset_prefix, sample_list)
    
# manual override:
# samples_dict = {"FeAu_0p5_tR_nscope": ["top_100um", "top_150um"]}
    
# now we have our samples_dict, we can process our data:

for sample, datasets in samples_dict.items():
    for dataset in datasets:
        print(f"Processing dataset {dataset} in sample {sample}")
        dset_path = os.path.join(ds.analysisroot, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")
        if not os.path.exists(dset_path):
            print(f"Missing DataSet file for {dataset} in sample {sample}, skipping")
            continue
        
        print("Importing DataSet object")
        
        ds = ImageD11.sinograms.dataset.load(dset_path)
        print(f"I have a DataSet {ds.dset} in sample {ds.sample}")
        
        if not os.path.exists(ds.refoutfile):
            print(f"Couldn't find PBP refinement output file for {dataset} in sample {sample}, skipping")
            continue
        
        if os.path.exists(os.path.join(ds.analysispath, 'pbp_tensormap_refined.h5')):
            print(f"Already have refined TensorMap output file for {dataset} in sample {sample}, skipping")
            continue
        
        refine = PBPRefine.from_h5(ds.refmanfile)
        refine.refinedmap.choose_best(min_unique)
        
        # first let's work out what phase we have
        phases = {0: ref_ucell}

        # let's make a phase id map from our pbpmap
        phase_ids = TensorMap.recon_order_to_map_order(np.where(refine.refinedmap.best_nuniq > min_unique, 0, -1))

        # reshape the fancy strain map too
        eps_sample = TensorMap.recon_order_to_map_order(refine.refinedmap.best_eps)

        tmap = TensorMap.from_pbpmap(refine.refinedmap, steps=(1, ds.ystep, ds.ystep), phases=phases)
        tmap['phase_ids'] = phase_ids
        tmap['eps_sample'] = eps_sample
        
        tmap.get_ipf_maps()
        eul = tmap.euler
        
        tmap.to_h5(os.path.join(ds.analysispath, 'pbp_tensormap_refined.h5'))
        tmap.to_paraview(os.path.join(ds.analysispath, 'pbp_tensormap_refined.h5'))
        ctf_path = os.path.join(ds.analysispath, 'pbp_tensormap_refined.ctf')
        tmap.to_ctf_mtex(ctf_path, z_index=0)

        ds.save()

print("Done!")